In [ ]:
from typing import Optional, Tuple, Dict, List
import numpy as np
import json
import math

# Qiskit imports
try:
    from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
    from qiskit.providers.aer import AerSimulator
    from qiskit.providers.aer.noise import NoiseModel, depolarizing_error, thermal_relaxation_error
    from qiskit.providers.aer.noise import pauli_error
    from qiskit.tools.monitor import job_monitor
    from qiskit.utils import QuantumInstance
    from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
    from qiskit.providers.ibmq import IBMQ
    from qiskit import execute
except Exception as e:
    raise ImportError(
        "This script requires Qiskit (qiskit, qiskit-aer). Install with: pip install qiskit qiskit-aer"
    ) from e



def galton_board_circuit(num_layers: int, encoding: str = 'path', qubit_prefix: str = 'q') -> QuantumCircuit:
    
    # number of qubits required: ceil(log2(num_layers+1)) to represent bins in binary
    n_bins = num_layers + 1
    n_qubits = math.ceil(math.log2(n_bins))
    q = QuantumRegister(n_qubits, name=qubit_prefix)
    c = ClassicalRegister(n_qubits, name='c')
    qc = QuantumCircuit(q, c)

    # Simple approach: start in |0...0>, apply a sequence of controlled rotations
    # to generate a binomial-like distribution across basis states. For exact
    # binomial distribution we'd implement coin flips (Hadamard) on a path register
    # but that requires additional ancilla. We trade-off circuit size vs fidelity.

    # Strategy: implement num_layers independent "coin" qubits logically, then
    # encode the count of |1>s into the output register using a reversible adder
    # (compact but expensive). Instead we use a simple tree of mixing unitaries
    # that approximate binomial amplitudes across the output basis.

    # Practical implementation (depth-friendly): apply single-qubit rotations on
    # each computational qubit to distribute amplitude. For small boards this performs well.

    for layer in range(num_layers):
        # apply parameterized rotation on each qubit to spread amplitude gradually
        # small angle rotations reduce entangling requirements but still approximate spread
        angle = math.pi / (2 * (layer + 1))  # heuristic
        for qb in range(n_qubits):
            qc.ry(angle, q[qb])
        # optionally insert light entanglement to mix amplitudes
        for qb in range(n_qubits - 1):
            qc.cx(q[qb], q[qb + 1])
            qc.rz(angle / 2, q[qb + 1])
            qc.cx(q[qb], q[qb + 1])

    qc.measure(q, c)
    return qc




def synthetic_noise_model() -> NoiseModel:
    """Create a simple synthetic noise model representative of superconducting
    qubits (typical T1/T2 and depolarizing gates)."""
    nm = NoiseModel()

    # single-qubit depolarizing
    p1 = 0.002  # 0.2% 1-qubit gate error
    p2 = 0.01   # 1% 2-qubit gate error
    single_error = depolarizing_error(p1, 1)
    two_error = depolarizing_error(p2, 2)

    nm.add_all_qubit_quantum_error(single_error, ['u1', 'u2', 'u3', 'rx', 'ry', 'rz', 'h'])
    nm.add_all_qubit_quantum_error(two_error, ['cx', 'cz'])

    # measurement error
    meas_error = pauli_error([('X', 0.01), ('I', 0.99)])  # simple model; replace with readout map if available
    nm.add_all_qubit_quantum_error(meas_error, ['measure'])

    return nm


def noise_model_from_ibmq_backend(backend_name: Optional[str] = None, token: Optional[str] = None) -> Tuple[Optional[NoiseModel], Optional[Dict]]:
    
    try:
        if token:
            IBMQ.enable_account(token)
        provider = IBMQ.load_account() if not token else IBMQ.get_provider(hub='ibm-q')

        if backend_name is None:
            # pick the least-loaded backend with sufficient qubits
            backends = provider.backends(filters=lambda b: (not b.configuration().simulator) and b.status().operational)
            backend = sorted(backends, key=lambda b: b.status().pending_jobs)[0]
        else:
            backend = provider.get_backend(backend_name)

        from qiskit.providers.aer.noise import NoiseModel
        noise_model = NoiseModel.from_backend(backend)
        props = backend.properties()
        return noise_model, props

    except Exception as e:
        print("Could not fetch IBMQ backend noise model:", str(e))
        return None, None




def measurement_error_mitigation(qc: QuantumCircuit, backend, shots=8192):

  
    # Build calibration circuits
    from qiskit.ignis.mitigation.measurement import complete_meas_cal

    qubits = list(range(qc.num_qubits))
    meas_calibs, state_labels = complete_meas_cal(qubits=qubits, qr=qc.qregs[0])

    job = execute(meas_calibs, backend=backend, shots=shots)
    result = job.result()
    meas_fitter = CompleteMeasFitter(result, state_labels)
    return meas_fitter


def distribution_distance(target: np.ndarray, observed: np.ndarray) -> float:
   
    # normalize
    t = target / np.sum(target)
    o = observed / np.sum(observed)
    return 0.5 * np.sum(np.abs(t - o))


def ideal_binomial_distribution(num_layers: int) -> np.ndarray:
    n = num_layers
    # bins 0..n inclusive
    probs = np.array([math.comb(n, k) for k in range(n + 1)], dtype=float)
    probs /= probs.sum()
    return probs

def run_sweep(max_layers: int = 8,
              shots: int = 4096,
              noise_model: Optional[NoiseModel] = None,
              backend_name: Optional[str] = None,
              ibmq_token: Optional[str] = None,
              optimization_levels: List[int] = [0, 1, 2, 3],
              try_ibmq: bool = False) -> Dict:

   
    results = {}

    # Prepare simulator backend
    aer_sim = AerSimulator() if noise_model is None else AerSimulator(noise_model=noise_model)

    for layers in range(1, max_layers + 1):
        target = ideal_binomial_distribution(layers)
        for opt in optimization_levels:
            qc = galton_board_circuit(layers)

            # transpile for simulator; for real-device mapping you'd transpile to backend
            tp_qc = transpile(qc, aer_sim, optimization_level=opt)
            depth = tp_qc.depth()

            # execute
            job = aer_sim.run(tp_qc, shots=shots)
            res = job.result()
            counts = res.get_counts()

            # Convert counts (binary strings) to bin counts (0..layers)
            n_qubits = math.ceil(math.log2(layers + 1))
            bins_counts = np.zeros(layers + 1)
            for bitstr, v in counts.items():
                # qiskit returns little-endian by default in counts (rightmost is q0)
                # we normalize by taking integer value
                intval = int(bitstr.replace(' ', ''), 2)
                if intval <= layers:
                    bins_counts[intval] += v
                # else: overflow states map to nearest or ignored

            tvd = distribution_distance(target, bins_counts)

            results[(layers, opt)] = {
                'layers': layers,
                'opt_level': opt,
                'depth': depth,
                'tvd': float(tvd),
                'counts': bins_counts.tolist(),
                'raw_counts': counts
            }

    return results


    summary = {}
    for (layers, opt), data in results.items():
        summary.setdefault(layers, [])
        summary[layers].append(data)

    best_by_layer = {}
    for layers, entries in summary.items():
        # choose entry with minimum TVD; tie-breaker: smaller depth
        best = min(entries, key=lambda e: (e['tvd'], e['depth']))
        best_by_layer[layers] = best

    # choose maximum layers with TVD below a threshold (e.g., 0.15)
    good_layers = [L for L, v in best_by_layer.items() if v['tvd'] <= 0.15]
    max_good = max(good_layers) if good_layers else None

    return {
        'best_by_layer': best_by_layer,
        'max_layers_with_acceptable_tvd': max_good,
        'threshold': 0.15
    }


if _name_ == '_main_':
    import argparse
    parser = argparse.ArgumentParser(description='Optimized Galton Board Simulation with Noise')
    parser.add_argument('--max_layers', type=int, default=8, help='Max number of Galton board layers to test')
    parser.add_argument('--shots', type=int, default=4096)
    parser.add_argument('--use_ibmq', action='store_true', help='Attempt to load IBMQ noise model (requires credentials)')
    parser.add_argument('--ibmq_token', type=str, default=None, help='IBMQ API token (optional if saved)')
    parser.add_argument('--backend_name', type=str, default=None, help='IBMQ backend name (optional)')
    parser.add_argument('--save', type=str, default='galton_results.json')
    args = parser.parse_args()

    # load noise model
    noise_model = None
    props = None
    if args.use_ibmq:
        nm, props = noise_model_from_ibmq_backend(args.backend_name, args.ibmq_token)
        if nm is not None:
            noise_model = nm
        else:
            print('Falling back to synthetic noise model')
            noise_model = synthetic_noise_model()
    else:
        noise_model = synthetic_noise_model()

    results = run_sweep(max_layers=args.max_layers, shots=args.shots, noise_model=noise_model)
    analysis = analyze_results(results)

    out = {
        'results': results,
        'analysis': analysis
    }

    with open(args.save, 'w') as f:
        json.dump(out, f, indent=2)

    print(f"Sweep complete. Saved to {args.save}.")
    print('Best summary:')
    if analysis['max_layers_with_acceptable_tvd'] is not None:
        print(f"Max layers with TVD <= {analysis['threshold']}: {analysis['max_layers_with_acceptable_tvd']}")
    else:
        print('No layer achieved threshold')